In [2]:
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
import matplotlib.colors as colors
import pandas as pd
import math
from sklearn.linear_model import Lasso as Lasso_Reg
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score as val_score
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split as sk_split
import statsmodels.api as sm

%matplotlib inline

In [3]:
#read in crops
allcrops_2000 = pd.read_csv('datasets/allcrops_2000.csv')
# names of all the crops
crop_names = allcrops_2000['ItemName'].unique()

# remove nan
crop_names = crop_names[:-1]
# list to put our separated crop dataframes
all_crop_dfs = []

# find each crop-separated df and put it in the list of dataframes
for crop in crop_names:
    crop_df = allcrops_2000[(allcrops_2000['ItemName'] == crop)]
    all_crop_dfs.append(crop_df)

print crop_names

['Wheat and products' 'Barley and products' 'Maize and products'
 'Rye and products' 'Oats' 'Cereals, Other' 'Potatoes and products'
 'Sugar (Raw Equivalent)' 'Sweeteners, Other' 'Pulses, Other and products'
 'Nuts and products' 'Soyabeans' 'Groundnuts (Shelled Eq)'
 'Coconuts - Incl Copra' 'Sesame seed' 'Olives (including preserved)'
 'Oilcrops, Other' 'Soyabean Oil' 'Groundnut Oil' 'Sunflowerseed Oil'
 'Rape and Mustard Oil' 'Cottonseed Oil' 'Palmkernel Oil' 'Coconut Oil'
 'Sesameseed Oil' 'Olive Oil' 'Maize Germ Oil' 'Oilcrops Oil, Other'
 'Tomatoes and products' 'Onions' 'Vegetables, Other' 'Oranges, Mandarines'
 'Lemons, Limes and products' 'Grapefruit and products' 'Citrus, Other'
 'Bananas' 'Apples and products' 'Pineapples and products' 'Dates'
 'Grapes and products (excl wine)' 'Fruits, Other' 'Coffee and products'
 'Cocoa Beans and products' 'Tea (including mate)' 'Pepper' 'Pimento'
 'Spices, Other' 'Wine' 'Beer' 'Beverages, Fermented'
 'Beverages, Alcoholic' 'Infant food' 'R

In [4]:
# Do the same as above, but for livestock
livestock_2000 = pd.read_csv('datasets/livestock_2000.csv')
# list of livestock names
livestock_names_all = livestock_2000['ItemName'].unique()
livestock_names_all = livestock_names_all[1:-1]
livestock_names = []
for name in livestock_names_all:
    if name != 'Grand Total':
        livestock_names.append(name)

# list to put our separated dataframes
livestock_dfs = []

# separate dataframe based on itemname
for crop in livestock_names:
    crop_df = livestock_2000[(livestock_2000['ItemName'] == crop)]
    livestock_dfs.append(crop_df)

print livestock_names
print len(livestock_names)


['Bovine Meat', 'Mutton & Goat Meat', 'Pigmeat', 'Poultry Meat', 'Meat, Other', 'Offals, Edible', 'Fats, Animals, Raw', 'Milk, Whole', 'Butter, Ghee', 'Cheese', 'Cream', 'Eggs', 'Honey', 'Freshwater Fish', 'Demersal Fish', 'Pelagic Fish', 'Marine Fish, Other', 'Crustaceans', 'Cephalopods', 'Molluscs, Other', 'Aquatic Plants', 'Fish, Body Oil', 'Fish, Liver Oil', 'Milk - Excluding Butter', 'Meat', 'Offals', 'Animal fats', 'Fish, Seafood', 'Aquatic Products, Other', 'Animal Products', 'Aquatic Animals, Others', 'Whey', 'Meat Meal', 'Meat, Aquatic Mammals', 'Hides and skins']
35


In [5]:
# read in disease rates
column_headers = ["Country", "Year", "Mortality", "Mortality 2", "Mortality 3"]
diabetes = pd.read_csv('datasets/diabetes.csv', names = column_headers)
cardio = pd.read_csv('datasets/cardiovascular.csv', names = column_headers)
cancer= pd.read_csv('datasets/Cancer.csv', names = column_headers)

In [6]:
cardio.iloc[3:, 0:3].head(n=5)
#cardio.columns = ['Country', 'Year', 'Mortality']

,Country,Year,Mortality
3,Afghanistan,2012,511.5
4,Afghanistan,2000,542.3
5,Albania,2012,436.2
6,Albania,2000,560.5
7,Algeria,2012,396.6


In [7]:
cardio_2012 = cardio[(cardio["Year"] == "2012")] 
cardio_2000 = cardio[(cardio["Year"] == "2000")] 

In [8]:
diabetes_2012 = diabetes[(diabetes['Year'] == '2012')] 
diabetes_2000 = diabetes[(diabetes['Year'] == '2000')] 
diabetes_2000

,Country,Year,Mortality,Mortality 2,Mortality 3
4,Afghanistan,2000,34.0,35.1,32.8
6,Albania,2000,8.3,8.2,8.3
8,Algeria,2000,67.2,62.1,73.4
10,Angola,2000,35.0,36.4,33.2
12,Argentina,2000,19.9,16.0,25.2
14,Armenia,2000,32.2,33.6,29.2
16,Australia,2000,10.2,8.0,13.0
18,Austria,2000,15.5,12.4,19.3
20,Azerbaijan,2000,19.6,21.4,16.6
22,Bahamas,2000,45.6,41.5,49.9


In [9]:
cancer_2012 = cancer[(cancer['Year'] == ' 2012')] 
cancer_2000 = cancer[(cancer['Year'] == ' 2000')] 
cancer_2012

,Country,Year,Mortality,Mortality 2,Mortality 3
3,Afghanistan,2012,123.6,110.6,142.2
5,Albania,2012,123.1,105.3,142.5
7,Algeria,2012,80.6,78.0,83.9
9,Angola,2012,89.6,86.1,97.9
11,Argentina,2012,131.5,109.1,166.5
13,Armenia,2012,219.6,175.3,277.1
15,Australia,2012,111.0,90.6,135.9
17,Austria,2012,122.0,99.7,152.3
19,Azerbaijan,2012,112.8,88.5,145.0
21,Bahamas,2012,103.8,88.1,131.8


In [10]:
cardio_2012.head(n=5)

,Country,Year,Mortality,Mortality 2,Mortality 3
3,Afghanistan,2012,511.5,520.9,498.0
5,Albania,2012,436.2,417.6,447.4
7,Algeria,2012,396.6,359.9,434.6
9,Angola,2012,374.6,369.8,376.4
11,Argentina,2012,191.7,148.4,247.8


In [11]:
cardio_2000.head(n=5)

,Country,Year,Mortality,Mortality 2,Mortality 3
4,Afghanistan,2000,542.3,554.1,523.1
6,Albania,2000,560.5,494.7,638.6
8,Algeria,2000,434.7,401.2,472.2
10,Angola,2000,383.4,385.2,375.2
12,Argentina,2000,226.6,169.1,302.8


In [12]:
all_foods = all_crop_dfs
print len(all_foods)

89


In [13]:
# plot for cardio
def food_cardio_plot(food_df, cardio):
    score = 1
    morts = []
    food_amts = []
    
    # for each country
    for country in food_df['AreaName'].values:
        # make sure it's in the disease df
        if not cardio[(cardio['Country'] == country)].empty:
            # find the disease data for that country
            country_data = cardio[(cardio['Country'] == country)]
            # find mortality rate
            mort = country_data.iloc[1,2]
            # append to mortality list
            morts.append(float(mort))
            # find amount of that particular food consumed in that country
            food_amt = food_df[(food_df['AreaName'] == country)].iloc[0, 9]
            # append to food_amounts
            food_amts.append(float(food_amt))
    
    # scatter the amounts against mortality rates
    #ax.scatter(food_amts, morts)
    
    # reshape for LinReg to get R^2
    X = np.array(food_amts).reshape(-1, 1)
    y = np.array(morts)
    
    if (len(food_df['ItemName'].values) > 0) and (X.shape[0] > 0):
        # linreg
        regr = LinReg()
        # fit, score linreg
        regr.fit(X, y)
        score = regr.score(X,y)
        #ax.set_title("R^2 = " + str(score))
    
    #ax.set_xlabel(food_df['ItemName'].values[0])
    #ax.set_ylabel('Cancer Disease Mortality Rate')
    
    return score
    

In [14]:
# Diabetes plot works the same way except for diabetes
def food_diabetes_plot(food_df, cardio):
    score = 1
    morts = []
    food_amts = []
    for country in food_df['AreaName'].values:
        if not cardio[(cardio['Country'] == country)].empty:
            country_data = cardio[(cardio['Country'] == country)]
            mort = country_data.iloc[1,2]
            morts.append(float(mort))
            food_amt = food_df[(food_df['AreaName'] == country)].iloc[0, 9]
            food_amts.append(float(food_amt))
    #ax.scatter( food_amts, morts)
    X = np.array(food_amts).reshape(-1, 1)
    y = np.array(morts)
    if (len(food_df['ItemName'].values) > 0) and (X.shape[0] > 0):
        regr = LinReg()
        regr.fit(X, y)
        score = regr.score(X,y)
        #ax.set_title("R^2 = " + str(score))
    
    #ax.set_xlabel(food_df['ItemName'].values[0] + str('(kg/capita/yr)'))
    #ax.set_ylabel('Diabetes Disease Mortality Rate')
    
    return score

In [15]:
# Cancer works the same way except for cancer
def food_cancer_plot(food_df, cardio):
    score = 1
    morts = []
    food_amts = []
    for country in food_df['AreaName'].values:
        if not cardio[(cardio['Country'] == country)].empty:
            country_data = cardio[(cardio['Country'] == country)]
            mort = country_data.iloc[1,2]
            morts.append(float(mort))
            food_amt = food_df[(food_df['AreaName'] == country)].iloc[0, 9]
            food_amts.append(float(food_amt))
    #ax.scatter(food_amts, morts)
    
    X = np.array(food_amts).reshape(-1, 1)
    y = np.array(morts)
    
    if (len(food_df['ItemName'].values) > 0) and (X.shape[0] > 0):
        regr = LinReg()
        regr.fit(X, y)
        score = regr.score(X,y)
        #ax.set_title("R^2 = " + str(score))
        #ax.set_xlabel(food_df['ItemName'].values[0])
        #ax.set_ylabel('Cancer Disease Mortality Rate')
    return score

In [16]:
#fig, ax = plt.subplots(18, 5, figsize=(18, 50))

#flatten list of axes
#axes = [e for l in ax for e in l]

# store r2s
cardio_crop_r2 = {}

# go through and plot each one
for df in all_crop_dfs:
    r2 = food_cardio_plot(df, cardio)
    crop_name = df['ItemName'].values[0]
    cardio_crop_r2[crop_name] = r2

#plt.tight_layout()
#plt.show()

In [17]:
#fig, ax = plt.subplots(18, 5, figsize=(18, 50))

# flatten list of axes
#axes = [e for l in ax for e in l]

# store r2s
diabetes_crop_r2 = {}
    
# iterate through
for df in all_crop_dfs:
    r2 = food_diabetes_plot(df, diabetes)
    crop_name = df['ItemName'].values[0]
    diabetes_crop_r2[crop_name] = r2


#plt.tight_layout()
#plt.show()

In [18]:
#fig, ax = plt.subplots(18, 5, figsize=(18, 50))

# flatten list of axes
#axes = [e for l in ax for e in l]

# store r2s
cancer_crop_r2 = {}

# go through
for df in all_crop_dfs:
    r2 =food_cancer_plot(df, cancer)
    crop_name = df['ItemName'].values[0]
    cancer_crop_r2[crop_name] = r2

#plt.tight_layout()
#plt.show()

In [19]:
#fig, ax = plt.subplots(7, 5, figsize=(25, 35))

#axes = [e for l in ax for e in l]

cardio_livestock_r2 = {}

for df in livestock_dfs:
    r2 =food_cardio_plot(df, cardio)
    livestock_name = df['ItemName'].values[0]
    cardio_livestock_r2[livestock_name] = r2

#plt.tight_layout()
#plt.show()

In [20]:
#fig, ax = plt.subplots(7, 5, figsize=(25, 35))

#axes = [e for l in ax for e in l]

diabetes_livestock_r2 = {}

for df in livestock_dfs:
    r2 =food_diabetes_plot(df, diabetes)
    livestock_name = df['ItemName'].values[0]
    diabetes_livestock_r2[livestock_name] = r2

#plt.tight_layout()
#plt.show()

KeyboardInterrupt: 

In [ ]:
#fig, ax = plt.subplots(7, 5, figsize=(25, 35))

#axes = [e for l in ax for e in l]

cancer_livestock_r2 = {}

for df in livestock_dfs:
    r2 =food_cancer_plot(df, cancer)
    livestock_name = df['ItemName'].values[0]
    cancer_livestock_r2[livestock_name] = r2

print cancer_livestock_r2
#plt.tight_layout()
#plt.show()

In [ ]:
all_crop_dfs[0].head(n=5)

In [ ]:
# dictionary of all dataframes, crop and livestock
all_dfs = {}
for df in all_crop_dfs:
    crop_name = df['ItemName'].values[0]
    all_dfs[crop_name] = df

for df in livestock_dfs:
    crop_name = df['ItemName'].values[0]
    all_dfs[crop_name] = df

In [ ]:
print len(cancer_crop_r2)
print len(cancer_livestock_r2)
for k, v in cancer_livestock_r2.items():
    cancer_crop_r2[k] = v
print len(cancer_crop_r2)

In [ ]:

for k, v in diabetes_livestock_r2.items():
    diabetes_crop_r2[k] = v

In [ ]:
print len(cardio_crop_r2)
print len(cardio_livestock_r2)
for k, v in cardio_livestock_r2.items():
    cardio_crop_r2[k] = v
print len(cardio_crop_r2)

In [ ]:
cancer_r2_sort = sorted(cancer_crop_r2.items(), key= lambda x: -x[1])
diabetes_r2_sort = sorted(diabetes_crop_r2.items(), key= lambda x: -x[1])
cardio_r2_sort = sorted(cardio_crop_r2.items(), key= lambda x: -x[1])

In [ ]:
print cardio_r2_sort

In [ ]:
print cancer_2000.shape
cancer_2000.head(n=5)

# Lin Reg for Cardio

In [ ]:
N = 15
#x = pd.DataFrame({}) 

countries = list(cancer_2000['Country'].values)
cols = []
for name, r2 in cardio_r2_sort[:N]:
    if r2 != 1.0 and name != 'Cottonseed Oil':
        df = all_dfs[name]
        print df.shape
        cols.append(name)
        for country in countries:
            if country not in df['AreaName'].values:
                countries.remove(country)
print len(countries)

x = []
for country in countries:
    predictors = []
    for name, r2 in cardio_r2_sort[:N]:
        if r2 != 1.0 and name != 'Cottonseed Oil':
            df = all_dfs[name]
            for index, area in enumerate(df['AreaName'].values):
                if area == country and not math.isnan(df['Value'].values[index]):
                    predictors.append(df['Value'].values[index])
    x.append(predictors)

x_df = pd.DataFrame(x, columns = cols)
x_df.head(n=5)                 
    
        
    

In [ ]:
y = []
for country in countries:
    for index, area in enumerate(cancer_2000['Country'].values):
        if area == country:
            y.append(cancer_2000['Mortality'].values[index])
            
y_df = pd.DataFrame(y, columns = ['Cancer Mortality Rate'])
y_df.head(n=5)

In [ ]:
x_df.head(n=30)

In [ ]:
last = -1
linreg = LinReg()
#linreg.fit(x_df.iloc[:, :last], y_df)
#linreg.score(x_df.iloc[:, :last], y_df)
linreg.fit(x_df, y_df)
linreg.score(x_df, y_df)

# Lin Reg for Diabetes

In [ ]:
N = 22
#x = pd.DataFrame({}) 

countries = list(diabetes_2000['Country'].values)
cols = []
for name, r2 in diabetes_r2_sort[:N]:
    if r2 != 1.0 and name != 'Rye and products' and name != 'Rape and Mustard Oil':
        df = all_dfs[name]
        print df.shape
        cols.append(name)
        for country in countries:
            if country not in df['AreaName'].values:
                countries.remove(country)
print len(countries)
print cols
print len(cols)

x = []
for country in countries:
    predictors = []
    for name, r2 in diabetes_r2_sort[:N]:
        if r2 != 1.0 and name != 'Rye and products' and name != 'Rape and Mustard Oil':
            df = all_dfs[name]
            for index, area in enumerate(df['AreaName'].values):
                #if area == country and not math.isnan(df['Value'].values[index]):
                if area == country:
                    predictors.append(df['Value'].values[index])
                    #print name
                    break
    x.append(predictors)
    #print 'end'

x_df = pd.DataFrame(x, columns = cols)
x_df.head(n=5)                 
    
        

In [ ]:
y = []
for country in countries:
    for index, area in enumerate(diabetes_2000['Country'].values):
        if area == country:
            y.append(diabetes_2000['Mortality'].values[index])
            
y_df = pd.DataFrame(y, columns = ['Diabetes Mortality Rate'])
y_df.head(n=5)

In [ ]:
x_df.head(n=20)

In [ ]:
last = -1
linreg = LinReg()
#linreg.fit(x_df.iloc[:, :last], y_df)
#linreg.score(x_df.iloc[:, :last], y_df)
linreg.fit(x_df, y_df)
linreg.score(x_df, y_df)

# Lin Reg for Cancer

In [ ]:
N = 20
#x = pd.DataFrame({}) 

countries = list(cancer_2000['Country'].values)
#print countries
#print cancer_r2_sort[:20]
cols = []
for name, r2 in cancer_r2_sort[:N]:
    if r2 != 1.0 and name != 'Palm Oil' and name != 'Millet and products' and name != 'Rye and products':
        df = all_dfs[name]
        print df.shape
        cols.append(name)
        for country in countries:
            if country not in df['AreaName'].values:
                countries.remove(country)
    else:
        print 'r^2 was 1'
print len(countries)
print cols
print len(cols)

x = []
for country in countries:
    predictors = []
    for name, r2 in cancer_r2_sort[:N]:
        if r2 != 1.0 and name != 'Palm Oil' and name != 'Millet and products' and name != 'Rye and products':
            df = all_dfs[name]
            for index, area in enumerate(df['AreaName'].values):
                #if area == country and not math.isnan(df['Value'].values[index]):
                if area == country:
                    predictors.append(df['Value'].values[index])
                    #print name
                    break
    x.append(predictors)
    #print 'end'

x_df = pd.DataFrame(x, columns = cols)
x_df.head(n=5)                 
    
        

In [ ]:
y = []
for country in countries:
    for index, area in enumerate(cancer_2000['Country'].values):
        if area == country:
            y.append(cancer_2000['Mortality'].values[index])
            
y_df = pd.DataFrame(y, columns = ['Cancer Mortality Rate'])
y_df.head(n=5)

In [ ]:
x_df.head(n=20)

In [ ]:
last = -1
linreg = LinReg()
#linreg.fit(x_df.iloc[:, :last], y_df)
#linreg.scoare(x_df.iloc[:, :last], y_df)
linreg.fit(x_df, y_df)
linreg.score(x_df, y_df)

# Looking at missing data

In [21]:
print len(cardio_2000['Country'].unique())
print len(diabetes_2000['Country'].unique())
print len(cancer_2000['Country'].unique())

172
172
172


In [22]:
cardio_lands = cardio_2000['Country'].unique()
diabetes_lands = diabetes_2000['Country'].unique()
cancer_lands = cancer_2000['Country'].unique()

In [24]:
df = pd.read_csv('datasets/allwregions_92_00.csv')
df.head(n=5)

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,CC,Food Supply - Crops Primary Equivalent,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2617,Apples and products,1992,1992,kg,1.08,Fc,Calculated data
1,CC,Food Supply - Crops Primary Equivalent,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2617,Apples and products,1993,1993,kg,1.00,Fc,Calculated data
2,CC,Food Supply - Crops Primary Equivalent,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2617,Apples and products,1994,1994,kg,0.87,Fc,Calculated data
3,CC,Food Supply - Crops Primary Equivalent,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2617,Apples and products,1995,1995,kg,0.69,Fc,Calculated data
4,CC,Food Supply - Crops Primary Equivalent,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2617,Apples and products,1996,1996,kg,0.87,Fc,Calculated data


In [25]:
all_countries = df['Country'].unique()
len(all_countries)
print df.columns
print type(df['Year'].values[0])

Index([u'Domain Code', u'Domain', u'Country Code', u'Country', u'Element Code',
       u'Element', u'Item Code', u'Item', u'Year Code', u'Year', u'Unit',
       u'Value', u'Flag', u'Flag Description'],
      dtype='object')
<type 'numpy.int64'>


In [26]:
all_crops = df['Item'].unique()
print len(all_crops)
c180 = 0
c170 = 0
c150 = 0

#crop_countries = set()

for crop in all_crops:
    countries = set()
    c_2000 = set()
    for i, country in enumerate(df['Country'].values):
        if country in cancer_lands:
            if df['Item'].values[i] == crop:
                countries.add(country)
                if df['Year'].values[i] == 2000:
                    c_2000.add(country)
    print crop, len(countries), len(c_2000)
    count = len(countries)
    if count == 180:
        c180 += 1
    elif count >= 170:
        c170 += 1
    elif count >= 150:
        c150 += 1
print c180, c170, c150
        #if df['Year'].values[i] == year:
            
    #for i in range(11):
        #year = i+1990
        
                
        

75
Apples and products 151 151
Bananas 143 143
Barley and products 139 139
Beer 150 150
Beverages, Alcoholic 148 148
Cereals, Other 141 141
Citrus, Other 134 134
Cocoa Beans and products 151 151
Coconuts - Incl Copra 145 145
Coffee and products 151 151
Cottonseed Oil 101 101
Dates 137 137
Fruits, Other 151 151
Grapes and products (excl wine) 149 149
Groundnut Oil 141 141
Infant food 138 138
Maize and products 147 147
Millet and products 59 59
Nuts and products 149 149
Oilcrops Oil, Other 145 145
Olive Oil 148 148
Olives (including preserved) 136 136
Oranges, Mandarines 150 150
Palm Oil 109 109
Pepper 146 146
Pineapples and products 145 145
Potatoes and products 151 151
Pulses, Other and products 147 147
Rape and Mustard Oil 109 109
Rice (Milled Equivalent) 151 151
Rice (Paddy Equivalent) 151 151
Roots & Tuber Dry Equiv 151 151
Sesame seed 105 105
Sesameseed Oil 113 113
Soyabean Oil 148 148
Spices, Other 149 149
Sugar (Raw Equivalent) 151 151
Sugar, Raw Equivalent 151 151
Sugar, Refined

In [27]:
l = []
for i, country in enumerate(all_countries):
    if country not in cancer_lands:
        l.append(country)
print len(l)
print l

55
['Antigua and Barbuda', 'Belgium-Luxembourg', 'Bermuda', 'China, Hong Kong SAR', 'China, Macao SAR', 'China, mainland', 'China, Taiwan Province of', "C\xc3\xb4te d'Ivoire", 'Czechia', 'Czechoslovakia', 'Dominica', 'Ethiopia PDR', 'French Polynesia', 'Grenada', 'Kiribati', 'Netherlands Antilles (former)', 'New Caledonia', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Samoa', 'Sao Tome and Principe', 'Serbia and Montenegro', 'Sudan (former)', 'The former Yugoslav Republic of Macedonia', 'United Kingdom', 'Vanuatu', 'World', 'Africa', 'Eastern Africa', 'Middle Africa', 'Northern Africa', 'Southern Africa', 'Western Africa', 'Americas', 'Northern America', 'Central America', 'Caribbean', 'South America', 'Asia', 'Central Asia', 'Eastern Asia', 'Southern Asia', 'South-Eastern Asia', 'Western Asia', 'Europe', 'Eastern Europe', 'Northern Europe', 'Southern Europe', 'Western Europe', 'Oceania', 'Australia & New Zealand', 'Melanesia', 'Micronesia', 'Polynesia']

In [28]:
for i, country in enumerate(cancer_lands):
    if country not in all_countries:
        print country

Bahrain
Bhutan
Burundi
Cote d'Ivoire
Comoros
Czech Republic
Democratic Republic of the Congo
Equatorial Guinea
Eritrea
Libya
Montenegro
Papua New Guinea
Qatar
Serbia
Singapore
Somalia
South Sudan
Sudan
Syrian Arab Republic
The former Yugoslav republic of Macedonia
United Kingdom of Great Britain and Northern Ireland


### Creates dictionary where each key is a crop whose value is a dictionary with countries as keys and values as the years present in the data

In [80]:

#crop_countries = set()
cropdata = {}
for crop in all_crops:
    data = {}
    for i, country in enumerate(df['Country'].values):
        count = 0
        if country in cancer_lands:
            data[country] = []
    for i, country in enumerate(df['Country'].values):
        if df['Item'].values[i] == crop:
            if country in data:
                if len(data[country]) > 0:
                    data[country].append(df['Year'].values[i])
                else:
                    data[country] = [df['Year'].values[i]]
    #for country in data:
        #if len(data[country]) != 9:
            #print crop, country, data[country]
    cropdata[crop] = data

In [ ]:
print cropdata['Molasses']

### Next 2 cells: Creates a dictionary with countries as keys and values as the corresponding region

In [35]:
def parse_country(dataset): 
    country_full = pd.read_csv(str(dataset))
    country_array = np.unique(country_full["Country"])
    return country_array

In [36]:
import glob
path = 'datasets/countries/' 
allFiles = glob.glob(path + "/*.csv")
country_dictionary = {}
for file in allFiles:
    area_name = file.split('/')[2].split('.')[0]
    country_list = parse_country(file)
    for country in country_list:
        country_dictionary[country] = area_name
country_dictionary['China'] = 'Eastern Asia'
print country_dictionary

{'Canada': 'Northern America', 'Sao Tome and Principe': 'Middle Africa', 'Turkmenistan': 'Central Asia', 'Serbia and Montenegro': 'Southern Europe', 'Lithuania': 'Northern Europe', 'Cambodia': 'South-Eastern Asia', 'Ethiopia': 'Eastern Africa', 'Sri Lanka': 'Southern Asia', 'Swaziland': 'Southern Africa', 'Argentina': 'South America', 'Cameroon': 'Middle Africa', 'Burkina Faso': 'Western Africa', 'Ghana': 'Western Africa', 'Saudi Arabia': 'Western Asia', 'Slovenia': 'Southern Europe', 'Guatemala': 'Central America', 'Bosnia and Herzegovina': 'Southern Europe', 'Kuwait': 'Western Asia', 'Russian Federation': 'Eastern Europe', 'Germany': 'Western Europe', 'Dominica': 'Caribbean', 'Liberia': 'Western Africa', 'Maldives': 'Southern Asia', 'Pakistan': 'Southern Asia', 'Oman': 'Western Asia', 'Cabo Verde': 'Western Africa', 'Gabon': 'Middle Africa', 'New Zealand': 'Australia & New Zealand', 'Yemen': 'Western Asia', 'Jamaica': 'Caribbean', 'Albania': 'Southern Europe', 'Samoa': 'Polynesia', '

In [37]:
regions = set()
for land in country_dictionary:
    regions.add(country_dictionary[land])
print regions

set(['Caribbean', 'Middle Africa', 'Southern Europe', 'Micronesia', 'Western Africa', 'Central Asia', 'South-Eastern Asia', 'Australia & New Zealand', 'Eastern Africa', 'Northern Europe', 'Eastern Asia', 'Northern America', 'Polynesia', 'Southern Africa', 'Western Asia', 'Southern Asia', 'Melanesia', 'Central America', 'Northern Africa', 'Eastern Europe', 'South America', 'Western Europe'])


In [38]:
regions_df = pd.read_csv('datasets/onlyregions_92_00.csv')
regions_df.tail(n=5)

,Domain,Country,Element,Item,Year,Unit,Value
13694,Food Supply - Crops Primary Equivalent,Polynesia,Food supply quantity (kg/capita/yr),Yams,1996,kg,3.46
13695,Food Supply - Crops Primary Equivalent,Polynesia,Food supply quantity (kg/capita/yr),Yams,1997,kg,4.09
13696,Food Supply - Crops Primary Equivalent,Polynesia,Food supply quantity (kg/capita/yr),Yams,1998,kg,4.49
13697,Food Supply - Crops Primary Equivalent,Polynesia,Food supply quantity (kg/capita/yr),Yams,1999,kg,5.53
13698,Food Supply - Crops Primary Equivalent,Polynesia,Food supply quantity (kg/capita/yr),Yams,2000,kg,6.23


In [39]:
regions_df['Country'].unique()

array(['Eastern Africa', 'Middle Africa', 'Northern Africa',
       'Southern Africa', 'Western Africa', 'Northern America',
       'Central America', 'Caribbean', 'South America', 'Central Asia',
       'Eastern Asia', 'Southern Asia', 'South-Eastern Asia',
       'Western Asia', 'Eastern Europe', 'Northern Europe',
       'Southern Europe', 'Western Europe', 'Australia & New Zealand',
       'Melanesia', 'Micronesia', 'Polynesia'], dtype=object)

### Creates a dictionary with the regions as keys and the values as dictionaries with crops as keys and lists of (year, value) tuples as values

In [40]:
regiondata = {}
for i, region in enumerate(regions_df['Country'].unique()):
    regiondata[region] = {}

for i, region in enumerate(regions_df['Country'].values):
    regiondict = regiondata[region]
    crop = regions_df['Item'].values[i]
    year = regions_df['Year'].values[i]
    value = regions_df['Value'].values[i]
    if crop in regiondict:
        regiondata[region][crop].append((year, value))
    else:
        regiondata[region][crop] = [(year, value)]

In [ ]:
#print regiondata['Western Asia']

In [48]:
regions_dfL = pd.read_csv('datasets/onlyregionsL_92_00.csv')
regions_dfL.tail(n=5)

,Domain,Country,Element,Item,Year,Unit,Value
5134,Food Supply - Livestock and Fish Primary Equiv...,Polynesia,Food supply quantity (kg/capita/yr),Whey,1996,kg,0.01
5135,Food Supply - Livestock and Fish Primary Equiv...,Polynesia,Food supply quantity (kg/capita/yr),Whey,1997,kg,0.05
5136,Food Supply - Livestock and Fish Primary Equiv...,Polynesia,Food supply quantity (kg/capita/yr),Whey,1998,kg,0.02
5137,Food Supply - Livestock and Fish Primary Equiv...,Polynesia,Food supply quantity (kg/capita/yr),Whey,1999,kg,0.01
5138,Food Supply - Livestock and Fish Primary Equiv...,Polynesia,Food supply quantity (kg/capita/yr),Whey,2000,kg,0.02


In [75]:
dfL = pd.read_csv('datasets/noregionsL_92_00.csv')
dfL.tail(n=5)

,Domain,Country,Element,Item,Year,Unit,Value
38399,Food Supply - Livestock and Fish Primary Equiv...,China,Food supply quantity (kg/capita/yr),Whey,1996,kg,0.0
38400,Food Supply - Livestock and Fish Primary Equiv...,China,Food supply quantity (kg/capita/yr),Whey,1997,kg,0.0
38401,Food Supply - Livestock and Fish Primary Equiv...,China,Food supply quantity (kg/capita/yr),Whey,1998,kg,0.0
38402,Food Supply - Livestock and Fish Primary Equiv...,China,Food supply quantity (kg/capita/yr),Whey,1999,kg,0.0
38403,Food Supply - Livestock and Fish Primary Equiv...,China,Food supply quantity (kg/capita/yr),Whey,2000,kg,0.0


In [76]:
all_ls = dfL['Item'].unique()
print len(all_ls)

28


### Same thing as cropdata but with livestock

In [31]:

#crop_countries = set()
lsdata = {}
for ls in all_ls:
    data = {}
    for i, country in enumerate(dfL['Country'].values):
        count = 0
        if country in cancer_lands:
            data[country] = []
    for i, country in enumerate(dfL['Country'].values):
        if dfL['Item'].values[i] == ls:
            if country in data:
                if len(data[country]) > 0:
                    data[country].append(dfL['Year'].values[i])
                else:
                    data[country] = [dfL['Year'].values[i]]
    #for country in data:
        #if len(data[country]) != 9:
            #print crop, country, data[country]
    lsdata[ls] = data

### Same thing as regiondata but for livestock

In [44]:
regiondataL = {}
for i, region in enumerate(regions_dfL['Country'].unique()):
    regiondataL[region] = {}

for i, region in enumerate(regions_dfL['Country'].values):
    regiondict = regiondataL[region]
    ls = regions_dfL['Item'].values[i]
    year = regions_dfL['Year'].values[i]
    value = regions_dfL['Value'].values[i]
    if ls in regiondict:
        regiondataL[region][ls].append((year, value))
    else:
        regiondataL[region][ls] = [(year, value)]

In [45]:
regiondataL['Western Asia']

{'Aquatic Animals, Others': [(1992, 0.01),
  (1993, 0.01),
  (1994, 0.01),
  (1995, 0.01),
  (1996, 0.0),
  (1997, 0.0),
  (1998, 0.01),
  (1999, 0.0),
  (2000, 0.01)],
 'Aquatic Plants': [(1992, 0.0),
  (1993, 0.0),
  (1994, 0.0),
  (1995, 0.0),
  (1996, 0.0),
  (1997, 0.0),
  (1998, 0.0),
  (1999, 0.0),
  (2000, 0.0)],
 'Bovine Meat': [(1992, 6.1500000000000004),
  (1993, 5.9800000000000004),
  (1994, 5.8399999999999999),
  (1995, 6.21),
  (1996, 5.5599999999999996),
  (1997, 6.1299999999999999),
  (1998, 5.7999999999999998),
  (1999, 5.6699999999999999),
  (2000, 5.9800000000000004)],
 'Butter, Ghee': [(1992, 1.51),
  (1993, 1.5),
  (1994, 1.46),
  (1995, 1.5),
  (1996, 1.6100000000000001),
  (1997, 1.51),
  (1998, 1.4299999999999999),
  (1999, 1.3999999999999999),
  (2000, 1.3500000000000001)],
 'Cephalopods': [(1992, 0.059999999999999998),
  (1993, 0.050000000000000003),
  (1994, 0.050000000000000003),
  (1995, 0.050000000000000003),
  (1996, 0.070000000000000007),
  (1997, 0.0700

In [34]:
#lsdata['Whey']

{'Afghanistan': [],
 'Albania': [],
 'Algeria': [1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000],
 'Angola': [],
 'Argentina': [1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000],
 'Armenia': [],
 'Australia': [],
 'Austria': [],
 'Azerbaijan': [],
 'Bahamas': [1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000],
 'Bangladesh': [],
 'Barbados': [],
 'Belarus': [],
 'Belgium': [],
 'Belize': [],
 'Benin': [],
 'Bolivia (Plurinational State of)': [],
 'Bosnia and Herzegovina': [],
 'Botswana': [],
 'Brazil': [],
 'Brunei Darussalam': [],
 'Bulgaria': [1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000],
 'Burkina Faso': [1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000],
 'Cabo Verde': [1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000],
 'Cambodia': [1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000],
 'Cameroon': [1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000],
 'Canada': [1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000],
 'Central African Republic': 

In [72]:
df = pd.read_csv('datasets/noregions_92_00.csv')
df.head(n=5)

,Domain,Country,Element,Item,Year,Unit,Value
0,Food Supply - Crops Primary Equivalent,Afghanistan,Food supply quantity (kg/capita/yr),Apples and products,1992,kg,1.08
1,Food Supply - Crops Primary Equivalent,Afghanistan,Food supply quantity (kg/capita/yr),Apples and products,1993,kg,1.00
2,Food Supply - Crops Primary Equivalent,Afghanistan,Food supply quantity (kg/capita/yr),Apples and products,1994,kg,0.87
3,Food Supply - Crops Primary Equivalent,Afghanistan,Food supply quantity (kg/capita/yr),Apples and products,1995,kg,0.69
4,Food Supply - Crops Primary Equivalent,Afghanistan,Food supply quantity (kg/capita/yr),Apples and products,1996,kg,0.87


In [73]:
#df = df.drop('Domain', 1)
df.head(n=5)
df.tail(n=5)

,Domain,Country,Element,Item,Year,Unit,Value
96285,Food Supply - Crops Primary Equivalent,China,Food supply quantity (kg/capita/yr),Yams,1996,kg,0.0
96286,Food Supply - Crops Primary Equivalent,China,Food supply quantity (kg/capita/yr),Yams,1997,kg,0.0
96287,Food Supply - Crops Primary Equivalent,China,Food supply quantity (kg/capita/yr),Yams,1998,kg,0.0
96288,Food Supply - Crops Primary Equivalent,China,Food supply quantity (kg/capita/yr),Yams,1999,kg,0.0
96289,Food Supply - Crops Primary Equivalent,China,Food supply quantity (kg/capita/yr),Yams,2000,kg,0.0


### Functions to extract values from regiondata

In [74]:
def getvalue(region, crop, year):
    if crop in regiondata[region]:
        data = regiondata[region][crop]
        for yr, val in data:
            if yr == year:
                return val
    else:
        return 0

def getvalueL(region, ls, year):
    if ls in regiondataL[region]:
        data = regiondataL[region][ls]
        for yr, val in data:
            if yr == year:
                return val
    else:
        return 0

# Do not re run these cell

### They take awhile to run

### Filling in the data

In [77]:
import time
end = dfL.shape[0]
years = [1992 + i for i in range(9)]
domain = dfL['Domain'].values[0]
element = regions_dfL['Element'].values[0]
unit = dfL['Unit'].values[0]
check1 = time.time()
count = 0
for ls in lsdata:
    for country in lsdata[ls]:
        count += 1
        if len(lsdata[ls][country]) == 9:
            continue
        if len(lsdata[ls][country]) == 8:
            value = getvalueL(country_dictionary[country], ls, 1992)
            dfL.loc[end+1] = [domain, country, element, ls, year, unit, value]
            end += 1
        if len(lsdata[ls][country]) == 1:
            for i in range(1992, 2000):
                value = getvalueL(country_dictionary[country], ls, i)
                dfL.loc[end] = [domain, country, element, ls, i, unit, value]
                end += 1
        if len(lsdata[ls][country]) == 0:
            for i in range(1992, 2001):
                value = getvalueL(country_dictionary[country], ls, i)
                dfL.loc[end] = [domain, country, element, ls, i, unit, value]
                end += 1
check2 = time.time()
print check2 - check1     

69.8499732018


In [81]:
end = df.shape[0]
years = [1992 + i for i in range(9)]
domain = df['Domain'].values[0]
element = regions_df['Element'].values[0]
unit = df['Unit'].values[0]
check1 = time.time()
count = 0
for crop in cropdata:
    for country in cropdata[crop]:
        count += 1
        if len(cropdata[crop][country]) == 9:
            continue
        if len(cropdata[crop][country]) == 8:
            value = getvalue(country_dictionary[country], crop, 1992)
            df.loc[end+1] = [domain, country, element, crop, year, unit, value]
            end += 1
        if len(cropdata[crop][country]) == 1:
            for i in range(1992, 2000):
                value = getvalue(country_dictionary[country], crop, i)
                df.loc[end] = [domain, country, element, crop, i, unit, value]
                end += 1
        if len(cropdata[crop][country]) == 0:
            for i in range(1992, 2001):
                value = getvalue(country_dictionary[country], crop, i)
                df.loc[end] = [domain, country, element, crop, i, unit, value]
                end += 1
check2 = time.time()
print check2 - check1     

665.452387094


In [78]:
dfL.tail(n=5)

,Domain,Country,Element,Item,Year,Unit,Value
43510,Food Supply - Livestock and Fish Primary Equiv...,Democratic People's Republic of Korea,Food supply quantity (kg/capita/yr),Cream,1996,kg,0.01
43511,Food Supply - Livestock and Fish Primary Equiv...,Democratic People's Republic of Korea,Food supply quantity (kg/capita/yr),Cream,1997,kg,0.01
43512,Food Supply - Livestock and Fish Primary Equiv...,Democratic People's Republic of Korea,Food supply quantity (kg/capita/yr),Cream,1998,kg,0.00
43513,Food Supply - Livestock and Fish Primary Equiv...,Democratic People's Republic of Korea,Food supply quantity (kg/capita/yr),Cream,1999,kg,0.01
43514,Food Supply - Livestock and Fish Primary Equiv...,Democratic People's Republic of Korea,Food supply quantity (kg/capita/yr),Cream,2000,kg,0.01


In [82]:
df.tail(n=5)

,Domain,Country,Element,Item,Year,Unit,Value
115017,Food Supply - Crops Primary Equivalent,Belgium,Food supply quantity (kg/capita/yr),Coffee and products,1995,kg,6.31
115018,Food Supply - Crops Primary Equivalent,Belgium,Food supply quantity (kg/capita/yr),Coffee and products,1996,kg,6.67
115019,Food Supply - Crops Primary Equivalent,Belgium,Food supply quantity (kg/capita/yr),Coffee and products,1997,kg,6.58
115020,Food Supply - Crops Primary Equivalent,Belgium,Food supply quantity (kg/capita/yr),Coffee and products,1998,kg,6.42
115021,Food Supply - Crops Primary Equivalent,Belgium,Food supply quantity (kg/capita/yr),Coffee and products,1999,kg,6.78


### Creating a dictionary with food names as keys and corresponding df as values

In [83]:
crop_names = df['Item'].unique()
#crop_names = crop_names[:-1]
# list to put our separated crop dataframes
all_food_dfs = {}

# find each crop-separated df and put it in the list of dataframes
for crop in crop_names:
    crop_df = df[(df['Item'] == crop)]
    all_food_dfs[crop] = crop_df

In [84]:
ls_names = dfL['Item'].unique()
for ls in ls_names:
    ls_df = dfL[(dfL['Item'] == ls)]
    all_food_dfs[ls] = ls_df

### List of all food names

In [85]:
all_names = list(crop_names) + list(ls_names)

In [86]:
df_vals = {}
for country in df['Country'].unique():
    if country in cancer_lands:
        data = {}
        for crop in all_names:
            df = all_food_dfs[crop]
            for i, name in enumerate(df['Country'].values):
                if name == country:
                    if crop in data:
                        data[crop].append((df['Year'].values[i], df['Value'].values[i]))
                    else:
                        data[crop] = [(df['Year'].values[i], df['Value'].values[i])]
        df_vals[country] = data
                        

In [87]:
for country in df_vals:
    print len(df_vals[country])

103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103
103


In [164]:
predictors = {}
for country in df_vals:
    data = {}
    for crop in df_vals[country]:
        if crop not in df_vals[country]:
            print country, crop
        else:
            year_data = df_vals[country][crop]
            total = 0
            if len(year_data) == 0:
                data[crop] = 0
            else:
                count = 0
                for yr, val in year_data:
                    if not isinstance(val, float) and not isinstance(val, int):
                        #print type(val), country, crop, yr, type(year_data), year_data
                        continue
                    else:
                        total += val
                        count += 1
                avg = float(total)/float(count)
                data[crop] = avg
    predictors[country] = data
        
        
        
            

In [ ]:
x = []
for country in countries:
    predictors = []
    for name, r2 in cancer_r2_sort[:N]:
        if r2 != 1.0 and name != 'Palm Oil' and name != 'Millet and products' and name != 'Rye and products':
            df = all_dfs[name]
            for index, area in enumerate(df['AreaName'].values):
                #if area == country and not math.isnan(df['Value'].values[index]):
                if area == country:
                    predictors.append(df['Value'].values[index])
                    #print name
                    break
    x.append(predictors)
    #print 'end'

x_df = pd.DataFrame(x, columns = cols)
x_df.head(n=5)  

In [89]:
x = []
cols = []
for crop in predictors['France']:
    cols.append(crop)
for country in predictors:
    x_vals = []
    for crop in predictors[country]:
        x_vals.append(predictors[country][crop])
    if len(x_vals) != 103:
        print len(x_vals)
    x.append(x_vals)
print len(x)
print len(x[0])
x_df = pd.DataFrame(x, columns = cols)
x_df.head(n=5)

151
103


,Ricebran Oil,Plantains,"Sugar, Raw Equivalent","Beverages, Alcoholic",Roots & Tuber Dry Equiv,Olives (including preserved),Cloves,"Fats, Animals, Raw",Millet and products,Coconuts - Incl Copra,...,Tomatoes and products,"Oilcrops Oil, Other",Pelagic Fish,"Butter, Ghee",Bananas,"Spices, Other","Meat, Aquatic Mammals",Sweet potatoes,"Oranges, Mandarines",Rice (Milled Equivalent)
0,0.00,0.603333,47.554444,1.558889,16.850000,0.532222,0.002222,6.007778,0.000000,1.112222,...,30.738889,0.102222,5.432222,2.918889,13.284444,0.297778,0.0,0.466667,47.530000,6.638889
1,0.00,0.000000,14.420000,0.327778,3.318889,0.001111,0.000000,0.953333,0.826667,0.000000,...,31.094444,0.203333,0.008889,1.820000,0.043333,0.004444,0.0,0.000000,1.604444,7.101111
2,0.00,0.000000,30.954444,5.077778,24.973333,0.024444,0.000000,3.285556,0.000000,0.254444,...,4.458889,2.487778,12.195556,3.835556,4.492222,0.100000,0.0,0.000000,4.731111,1.540000
3,0.03,0.583333,4.422222,1.601111,3.670000,0.000000,0.011111,0.813333,0.000000,0.766667,...,0.000000,0.098889,0.363333,0.002222,11.788889,0.006667,0.0,3.337778,4.556667,167.165556
4,0.00,26.812222,4.042222,0.047500,17.707778,0.000000,0.000000,0.238750,3.186250,0.000000,...,0.778750,0.405556,0.000000,0.198889,1.043750,0.051111,0.0,3.178750,0.402222,1.253333


In [90]:
cancer = []
for country in predictors:
    for i, place in enumerate(cancer_2000['Country'].values):
        if place == country:
            cancer.append(cancer_2000['Mortality'].values[i])
            
cancer_df = pd.DataFrame(cancer, columns = ['Cancer Mortality Rate'])
print cancer_df.shape
cancer_df.head(n=5)
            

(151, 1)


,Cancer Mortality Rate
0,142.1
1,104.9
2,154.6
3,110.9
4,100.7


In [200]:
cancerreg = LinReg()
cancerreg.fit(x_df, cancer_df)
cancerreg.score(x_df, cancer_df)
cancerreg.coef_
cancer_coefs = cancerreg.coef_[0]
cancerreg.coef_

array([[  2.59622194e+01,  -2.83643735e-01,   3.62074517e+00,
          3.24950393e+00,   3.83439089e+00,   1.28178033e+00,
          4.51037763e+01,   7.74756915e+00,   9.36269777e-03,
         -1.21100979e+00,  -1.53853443e+00,  -5.43087951e-01,
         -1.14389605e+01,   3.70398425e+00,  -2.96470785e-01,
         -9.32949464e-01,  -6.41711821e-01,  -4.79716527e-01,
         -6.13921630e+01,   1.51822621e-01,  -3.69113138e+00,
          9.93549219e+00,  -1.20985128e+01,   3.49920815e-01,
         -1.32741726e+01,  -2.75468393e-01,  -1.22397482e-01,
         -5.60413266e-01,   1.02514982e+00,  -2.33227016e+00,
         -5.56399735e-01,  -9.39555199e-01,   8.60366706e+01,
          2.21103405e+00,   3.08237011e+01,   3.80934817e+00,
         -1.47773030e+00,  -1.64219001e+01,  -7.56792456e-01,
         -1.00318070e+01,   1.21716967e+01,   1.16801603e+00,
         -5.72781859e-01,   9.58600721e+00,  -9.80433727e+00,
          2.30449176e+00,  -8.87531016e+00,   3.90353142e+00,
        

In [197]:
print max(cancer_coefs)
print list(cancer_coefs).index(max(cancer_coefs))
#print x_df.columns[-1]
#print list(diabetes_coefs[:-1]).index(max(diabetes_coefs[:-1]))
#print x_df.columns[list(diabetes_coefs[:-1]).index(max(diabetes_coefs[:-1]))]

1676.63328243
48


In [196]:
maxpredictors = []
print len(x_df.columns)
print len(x_df.iloc[0, :].values)
for i in range(x_df.shape[0]):
    preds = x_df.iloc[i, :].values
    maxpredictor, index = 0, 0
    for j, val in enumerate(preds):
        pred = val * cancer_coefs[j]
        if pred > maxpredictor:
            maxpredictor = pred
            index = j
    maxpredictors.append((country_indices[i], x_df.columns[index]))    

print maxpredictors

103
103
[('Canada', 'Rice (Paddy Equivalent)'), ('Turkmenistan', 'Rice (Paddy Equivalent)'), ('Lithuania', 'Rice (Paddy Equivalent)'), ('Cambodia', 'Rice (Paddy Equivalent)'), ('Ethiopia', 'Rice (Paddy Equivalent)'), ('Swaziland', 'Rice (Paddy Equivalent)'), ('Argentina', 'Rice (Paddy Equivalent)'), ('Cameroon', 'Rice (Paddy Equivalent)'), ('Burkina Faso', 'Rice (Paddy Equivalent)'), ('Ghana', 'Rice (Paddy Equivalent)'), ('Saudi Arabia', 'Rice (Paddy Equivalent)'), ('Japan', 'Rice (Paddy Equivalent)'), ('Slovenia', 'Rice (Paddy Equivalent)'), ('Guatemala', 'Rice (Paddy Equivalent)'), ('Bosnia and Herzegovina', 'Rice (Paddy Equivalent)'), ('Guinea', 'Rice (Paddy Equivalent)'), ('Russian Federation', 'Rice (Paddy Equivalent)'), ('Germany', 'Rice (Paddy Equivalent)'), ('Spain', 'Rice (Paddy Equivalent)'), ('Liberia', 'Rice (Paddy Equivalent)'), ('Maldives', 'Rice (Paddy Equivalent)'), ('Paraguay', 'Rice (Paddy Equivalent)'), ('Pakistan', 'Rice (Paddy Equivalent)'), ('Oman', 'Rice (Paddy E

In [184]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
cancerreg = Lasso(alpha = .00001, normalize=True)
cancerreg.fit(x_df, cancer_df)
print cancerreg.score(x_df, cancer_df)
coefficients = cancerreg.coef_
print 'hi'
print len(coefficients)
print len([i for i, item in enumerate(coefficients) if abs(item) >0])

cancerreg = LassoCV(alphas = [.01, .1], normalize=True)
#cancerlist = list(cancer_df)
cancerreg.fit(x_df, cancer_df.values.ravel())
print cancerreg.score(x_df, cancer_df)

0.806921512682
hi
103
101
0.591901219824


In [194]:
from sklearn.linear_model import Ridge
cancerreg = Ridge(alpha = .0001, normalize=True)
cancerreg.fit(x_df, cancer_df)
cancerreg.score(x_df, cancer_df)

In [92]:
diabetes = []
for country in predictors:
    for i, place in enumerate(diabetes_2000['Country'].values):
        if place == country:
            diabetes.append(diabetes_2000['Mortality'].values[i])
            
diabetes_df = pd.DataFrame(diabetes, columns = ['Diabetes Mortality Rate'])
print diabetes_df.shape
diabetes_df.head(n=5)
            

(151, 1)


,Diabetes Mortality Rate
0,14.0
1,19.8
2,5.0
3,15.9
4,28.9


In [198]:
diabetes_reg = LinReg()
diabetes_reg.fit(x_df, diabetes_df)
print diabetes_reg.score(x_df, diabetes_df)
print diabetes_reg.coef_
diabetes_coefs = diabetes_reg.coef_[0]
print max(diabetes_coefs)
print list(diabetes_coefs).index(max(diabetes_coefs))
print x_df.columns[-1]
print list(diabetes_coefs[:-1]).index(max(diabetes_coefs[:-1]))
print x_df.columns[list(diabetes_coefs[:-1]).index(max(diabetes_coefs[:-1]))]
for i, coef in enumerate(diabetes_coefs):
    if coef < -2000:
        print i
x_df.columns[48]

0.824653893157
[[ -6.48079659e+01   3.86815657e-02   6.25661010e-01   2.44406049e-01
   -2.54568975e-01  -3.73141680e+00  -4.74258660e+01   6.26347033e-01
   -2.47844677e-02   6.10020155e-01  -1.88833574e+00   1.24898468e-01
    2.82090488e+00  -1.58158321e+00  -1.25044880e-01  -7.44598156e-02
   -1.36520493e+00  -1.60872752e+00  -7.35106729e+01   2.31195821e-01
   -1.65255154e+00  -2.32337654e+00   7.08579219e+00   8.66905707e-01
    1.80292627e+01   8.40850735e-01  -9.18775158e-01   1.67602923e-01
    4.98619604e+00  -1.48639086e+00  -3.37991513e-01   5.77821719e-01
   -9.59994412e+02  -3.99177076e-01   6.45094593e+00  -2.66903330e+00
   -1.38032186e-01   1.33519571e+00  -3.08976497e-01  -4.19925603e+00
    7.22123858e+00  -3.25051963e-01  -3.27574920e-01  -2.04545345e+01
    7.49456836e-01   2.23249560e-01   7.03037313e+00   1.35913090e+00
   -2.02788304e+03   1.02105784e-01  -2.31124338e-01  -1.13427639e+00
   -3.47712649e+00  -2.52425342e+00   1.09218178e+00   2.17391888e+00
   -5

'Rice (Paddy Equivalent)'

In [199]:
diabetes_reg.coef_

array([[ -6.48079659e+01,   3.86815657e-02,   6.25661010e-01,
          2.44406049e-01,  -2.54568975e-01,  -3.73141680e+00,
         -4.74258660e+01,   6.26347033e-01,  -2.47844677e-02,
          6.10020155e-01,  -1.88833574e+00,   1.24898468e-01,
          2.82090488e+00,  -1.58158321e+00,  -1.25044880e-01,
         -7.44598156e-02,  -1.36520493e+00,  -1.60872752e+00,
         -7.35106729e+01,   2.31195821e-01,  -1.65255154e+00,
         -2.32337654e+00,   7.08579219e+00,   8.66905707e-01,
          1.80292627e+01,   8.40850735e-01,  -9.18775158e-01,
          1.67602923e-01,   4.98619604e+00,  -1.48639086e+00,
         -3.37991513e-01,   5.77821719e-01,  -9.59994412e+02,
         -3.99177076e-01,   6.45094593e+00,  -2.66903330e+00,
         -1.38032186e-01,   1.33519571e+00,  -3.08976497e-01,
         -4.19925603e+00,   7.22123858e+00,  -3.25051963e-01,
         -3.27574920e-01,  -2.04545345e+01,   7.49456836e-01,
          2.23249560e-01,   7.03037313e+00,   1.35913090e+00,
        

In [165]:
country_indices = []
index = 0
for country in predictors:
    country_indices.append(country)
print country_indices[:10]
print len(country_indices)

['Canada', 'Turkmenistan', 'Lithuania', 'Cambodia', 'Ethiopia', 'Swaziland', 'Argentina', 'Cameroon', 'Burkina Faso', 'Ghana']
151


In [171]:
maxpredictors = []
print len(x_df.columns)
print len(x_df.iloc[0, :].values)
for i in range(x_df.shape[0]):
    preds = x_df.iloc[i, :].values
    maxpredictor, index = 0, 0
    for j, val in enumerate(preds):
        pred = val * diabetes_coefs[j]
        if pred > maxpredictor:
            maxpredictor = pred
            index = j
    maxpredictors.append((country_indices[i], x_df.columns[index]))    

for country, crop in maxpredictors:
    if crop != 'Sugar, Refined Equiv' and crop != 'Rice (Milled Equivalent)':
        print country, crop

103
103


In [191]:
diabetes_reg = Lasso(alpha=.001, normalize=True)
diabetes_reg.fit(x_df, diabetes_df)
print diabetes_reg.score(x_df, diabetes_df)
diabetes_reg.coef_
for col in x_df.columns:
    print '"' + col + '"' + ', '

0.809141987567
"Ricebran Oil", 
"Plantains", 
"Sugar, Raw Equivalent", 
"Beverages, Alcoholic", 
"Roots & Tuber Dry Equiv", 
"Olives (including preserved)", 
"Cloves", 
"Fats, Animals, Raw", 
"Millet and products", 
"Coconuts - Incl Copra", 
"Eggs", 
"Vegetables, Other", 
"Sesame seed", 
"Marine Fish, Other", 
"Wine", 
"Pigmeat", 
"Apples and products", 
"Rape and Mustard Oil", 
"Fish, Liver Oil", 
"Maize and products", 
"Sweeteners, Other", 
"Fish, Body Oil", 
"Honey", 
"Demersal Fish", 
"Groundnuts (Shelled Eq)", 
"Barley and products", 
"Maize Germ Oil", 
"Beer", 
"Groundnut Oil", 
"Sugar non-centrifugal", 
"Pineapples and products", 
"Pulses, Other and products", 
"Sugar (Raw Equivalent)", 
"Meat, Other", 
"Molasses", 
"Pimento", 
"Palm Oil", 
"Oilcrops, Other", 
"Dates", 
"Oats", 
"Soyabeans", 
"Freshwater Fish", 
"Beans", 
"Sugar beet", 
"Cephalopods", 
"Mutton & Goat Meat", 
"Sesameseed Oil", 
"Grapes and products (excl wine)", 
"Rice (Paddy Equivalent)", 
"Beverages, Fermented"

In [94]:
cardio = []
for country in predictors:
    for i, place in enumerate(cardio_2000['Country'].values):
        if place == country:
            cardio.append(cardio_2000['Mortality'].values[i])
            
cardio_df = pd.DataFrame(cardio, columns = ['Cardio Mortality Rate'])
print cardio_df.shape
cardio_df.head(n=5)
            

(151, 1)


,Cardio Mortality Rate
0,150.8
1,724.4
2,373.9
3,222.5
4,180.4


In [189]:
cardio_reg = LinReg()
cardio_reg.fit(x_df, cardio_df)
cardio_reg.score(x_df, cardio_df)
cardio_coefs = cardio_reg.coef_[0]
cardio_coefs

array([  1.40816287e+02,   5.36298274e-01,  -9.44522931e+00,
         3.20981545e+00,  -4.86703518e+00,  -1.61816614e+01,
        -3.24805843e+02,  -1.99980905e+00,   1.20601695e+00,
         4.38928752e+00,  -4.24737986e+00,  -3.89874612e-01,
         7.87702229e+00,  -1.45513038e+00,  -6.65062858e-01,
        -2.48549699e-01,  -6.40420336e-01,  -9.92381694e+00,
        -4.67123409e+02,  -1.11162691e+00,   1.07300562e+01,
        -4.34260700e+01,   3.60929511e+01,  -5.54016373e-01,
         1.59178894e+01,  -3.27237817e-01,   4.52861740e+01,
        -6.82208858e-01,  -1.49393958e+01,   2.56052752e+00,
        -5.24584540e-01,  -8.27315584e+00,   2.74667049e+03,
         6.33794634e+00,  -1.17445997e+02,  -8.37077245e+00,
        -1.03250201e+01,  -2.71808620e+01,   4.77456499e-01,
        -5.38808983e-01,   1.26696239e+01,   1.19219779e+00,
        -2.18874810e+00,  -2.62730611e+02,  -9.85536827e+00,
         3.25150319e+00,   6.04250490e+01,  -2.85050059e+00,
         8.44309361e+02,

In [193]:
maxpredictors = []
print len(x_df.columns)
print len(x_df.iloc[0, :].values)
for i in range(x_df.shape[0]):
    preds = x_df.iloc[i, :].values
    maxpredictor, index = 0, 0
    for j, val in enumerate(preds):
        pred = val * cardio_coefs[j]
        if pred > maxpredictor:
            maxpredictor = pred
            index = j
    maxpredictors.append((country_indices[i], x_df.columns[index]))    

print maxpredictors
#for country, crop in maxpredictors:
    #if crop != 'Sugar, Refined Equiv' and crop != 'Rice (Milled Equivalent)':
        #print country, crop

103
103
[('Canada', 'Sugar (Raw Equivalent)'), ('Turkmenistan', 'Sugar (Raw Equivalent)'), ('Lithuania', 'Sugar (Raw Equivalent)'), ('Cambodia', 'Rice (Paddy Equivalent)'), ('Ethiopia', 'Sugar (Raw Equivalent)'), ('Swaziland', 'Sugar (Raw Equivalent)'), ('Argentina', 'Sugar (Raw Equivalent)'), ('Cameroon', 'Sugar (Raw Equivalent)'), ('Burkina Faso', 'Rice (Paddy Equivalent)'), ('Ghana', 'Rice (Paddy Equivalent)'), ('Saudi Arabia', 'Sugar (Raw Equivalent)'), ('Japan', 'Rice (Paddy Equivalent)'), ('Slovenia', 'Sugar (Raw Equivalent)'), ('Guatemala', 'Sugar (Raw Equivalent)'), ('Bosnia and Herzegovina', 'Sugar (Raw Equivalent)'), ('Guinea', 'Rice (Paddy Equivalent)'), ('Russian Federation', 'Sugar (Raw Equivalent)'), ('Germany', 'Sugar (Raw Equivalent)'), ('Spain', 'Sugar (Raw Equivalent)'), ('Liberia', 'Rice (Paddy Equivalent)'), ('Maldives', 'Sugar (Raw Equivalent)'), ('Paraguay', 'Sugar (Raw Equivalent)'), ('Pakistan', 'Sugar (Raw Equivalent)'), ('Oman', 'Sugar (Raw Equivalent)'), ('Ca

In [142]:

cardio_reg = Lasso(alpha=.00001, normalize=True)
cardio_reg.fit(x_df, cardio_df)
cardio_reg.score(x_df, cardio_df)

0.84455869353323476

# Do not run things beyond this point -----

In [226]:
end = df.shape[0]
years = [1992 + i for i in range(9)]
for crop in cropdata:
    for country in cropdata[crop]:
        if len(cropdata[crop][country]) != 9 and len(cropdata[crop][country]) != 8 and len(cropdata[crop][country]) != 0 and len(cropdata[crop][country]) != 1:
            print len(cropdata[crop]), country, crop
            

In [227]:
print cropdata['Molasses']

{'Canada': [], 'Turkmenistan': [], 'Lithuania': [], 'Cambodia': [], 'Ethiopia': [], 'Swaziland': [], 'Argentina': [], 'Cameroon': [], 'Burkina Faso': [], 'Ghana': [], 'Saudi Arabia': [], 'Slovenia': [], 'Guatemala': [], 'Bosnia and Herzegovina': [], 'Kuwait': [], 'Russian Federation': [], 'Jordan': [], 'Spain': [], 'Liberia': [], 'Maldives': [], 'Pakistan': [], 'Oman': [], 'Cabo Verde': [], 'Gabon': [], 'New Zealand': [], 'Yemen': [], 'Jamaica': [], 'Albania': [], 'United Arab Emirates': [], 'India': [], 'Azerbaijan': [], 'Lesotho': [], 'Kenya': [], 'Tajikistan': [], 'Turkey': [], 'Afghanistan': [], 'Bangladesh': [], 'Iran (Islamic Republic of)': [], 'Viet Nam': [], 'Mongolia': [], 'France': [], 'Rwanda': [], 'Slovakia': [], 'Peru': [], 'Norway': [], 'Malawi': [], 'Benin': [], 'Cuba': [], 'United States of America': [], 'Togo': [], 'China': [], 'Armenia': [], 'Timor-Leste': [], 'Dominican Republic': [], 'Bolivia (Plurinational State of)': [], 'Ukraine': [], 'Finland': [], 'Indonesia': 

In [217]:
import copy
import time
years = [1992 + i for i in range(9)]
end = df.shape[0]
#for crop in cropdata[:1]:
N = 0
count = 0
#crop = cropdata.keys()[N]
crop = 'Ricebran Oil'
#for crop in cropdata:
countrieslist = copy.copy(df['Country'].values)
domain = df['Domain'].values[0]
element = regions_df['Element'].values[0]
unit = df['Unit'].values[0]
print len(countrieslist)
check1 = time.time()
for index, country in enumerate(countrieslist[:100]):
    if country in cancer_lands:
        for year in years[:1]:
            if year not in cropdata[crop][country]:
                count += 1
                region = country_dictionary[country]
            
                data = regiondata[region]
                check2 = time.time()
                if crop not in data:
                    df.loc[end+1] = [domain, country, element, crop, year, unit, 0]
                    end += 1
                else:
                    for yr, val in data[crop]:
                        if yr == year:
                            df.loc[end+1] = [domain, country, element, crop, year, unit, val]
                            end += 1
                            break
check3 = time.time()

#print check3-check2
print check3-check1

96423
4.38123202324


In [216]:
years = [1992 + i for i in range(9)]
end = df.shape[0]
#for crop in cropdata[:1]:
N = 0
count = 0
crop = cropdata.keys()[N]
#crop = 'Ricebran Oil'
check1 = time.time()
for index, country in enumerate(df['Country'].values[:100]):
    if country in cancer_lands:
        for year in years[:1]:
            if year not in cropdata[crop][country]:
                count += 1
                region = country_dictionary[country]
                found = False
                for i, region_name in enumerate(regions_df['Country'].values):
                    if region_name == region:
                        if regions_df['Item'].values[i] == crop and regions_df['Year'].values[i] == year:
                            val = regions_df['Value'].values[i]
                            #temp = pd.DataFrame([])
                            df.loc[end+1] = [df['Domain'].values[0], country, regions_df['Element'].values[i], crop, year, regions_df['Unit'].values[i], val]
                            end += 1
                            found = True
                            break
                if not found:
                    df.loc[end+1] = [df['Domain'].values[0], country, regions_df['Element'].values[i], crop, year, regions_df['Unit'].values[i], 0]
                    end += 1
                        

check3 = time.time()
print check3 - check1

4.12499308586


In [178]:
df.tail(n=5)

,Country,Element,Item,Year,Unit,Value
135467,Albania,Food supply quantity (kg/capita/yr),Ricebran Oil,1992,kg,0.0
135468,Albania,Food supply quantity (kg/capita/yr),Ricebran Oil,1992,kg,0.0
135469,Albania,Food supply quantity (kg/capita/yr),Ricebran Oil,1992,kg,0.0
135470,Albania,Food supply quantity (kg/capita/yr),Ricebran Oil,1992,kg,0.0
135471,Albania,Food supply quantity (kg/capita/yr),Ricebran Oil,1992,kg,0.0
